# Limits
First, we establish a rate limiter for Binance. This will avoid overcharging the rate limiter and potentially hitting the rate limit that can ban your account.

In [1]:
from panzer.limits import BinanceRateLimiter

In [2]:
l = BinanceRateLimiter()

Rate limits: [{'rateLimitType': 'REQUEST_WEIGHT', 'interval': 'MINUTE', 'intervalNum': 1, 'limit': 6000}, {'rateLimitType': 'ORDERS', 'interval': 'SECOND', 'intervalNum': 10, 'limit': 100}, {'rateLimitType': 'ORDERS', 'interval': 'DAY', 'intervalNum': 1, 'limit': 200000}, {'rateLimitType': 'RAW_REQUESTS', 'interval': 'MINUTE', 'intervalNum': 5, 'limit': 61000}]


2024-09-13	 14:41:21     INFO Updated server-local time offset: 928 ms


In [3]:
l.get()

{'orders_limit_per_ten_seconds': 100,
 'orders_limit_per_day': 200000,
 'weight_limit_per_minute': 6000,
 'raw_limit_per_5_minutes': 61000,
 'server_time_offset': 928,
 'five_minutes_counts': {5754104: 2},
 'minutes_weights': {28770521: 21},
 'ten_seconds_orders_counts': {},
 'daily_orders_count': {}}

# Credentials
First, we add api keys and secret to the credential manager. If not, they will be asked by prompt. Adding with is_sensitive parameter will save the credentials securely. If not 
provided then it will be automatically secured if name of the credential is "api_key" or "api_secret".

In [4]:
from panzer.keys import CredentialManager

In [5]:
c = CredentialManager()

In [6]:
c.credentials

{}

## Cached credentials
If CredentialManager is asked for a credential that is already in the cache, it will return the cached value instead of asking for it again. Even it is not loaded in the object 
at first creation, it will add from disk cache automatically when needed, and it will prompt for it if it is not loaded. 

In [7]:
c.add("api_key", "test", is_sensitive=True)
# c.add("api_secret", "your_api_secret", is_string=True)

2024-09-13	 14:41:23     INFO Variable api_key exists in cache file. To renew it delete line in cache file at: C:\Users\hanca\panzer.tmp


'ymHfeSbzuL8gYqDPaH3yc0yl4NaMmRvOBVdbmYEWxpXwS/SaPht1uH3abKKTu3nbxBjF7we+d/6YeBknlCLfU2Vn5uqXbB4VZ2LyOvtWj84='

In [8]:
# cached in file can be retrieved with `file_manager` attribute.
c.file_manager

File at C:\Users\hanca\panzer.tmp:
# Archivo de credenciales de Panzer
api_secret = "uU7DyqKUkvOX4OsMpeBetxy05s2ZGz6CY0DgkOHhlvrbakh/fLU7K4MXY3IHBJbqs+qlznAVCmGR1fWzprNR42A1YsUpRK8B3m3xBFPZ40g="
postgresql_port = "5432"
postgresql_user = "nando"
postgresql_database = "crypto"
private_chat_id = "LrtmUpkUyr90wgFX2f6uQg=="
postgresql_password = "XQ48RaWPccRth30uGzqa2A=="
api_key = "ymHfeSbzuL8gYqDPaH3yc0yl4NaMmRvOBVdbmYEWxpXwS/SaPht1uH3abKKTu3nbxBjF7we+d/6YeBknlCLfU2Vn5uqXbB4VZ2LyOvtWj84="

In [9]:
c

{'api_key': 'jfUyii/sHV3DFhUxL8noew=='}

# Examples of API calls
Credentials for API calls are going to be asked by prompt the first time if they are not in the creds cache.


In [10]:
from panzer.request import get, post

## Get klines example
Assuming credentials are already in the cache, will check limits and then make the API call.

In [11]:
url = 'https://api.binance.com/api/v3/klines'
params = {
    "symbol": "BTCUSDT",  # Par BTCUSDT
    "interval": "1m",     # Intervalo de 1 minuto
    "limit": 1000            # Limitar a las últimas 5 velas
}

In [12]:
l.can_make_request(weight=1, is_order=False)

True

In [13]:
response, headers = get(params=params, 
                        url=url,
                        full_sign=False,
                        server_time_offset=l.server_time_offset,)

In [14]:
# first limits update will notify sync of the object with api server if they differ
l.update_from_headers(headers)

2024-09-13	 14:41:23  WARNING Rate limit x-mbx-used-weight-1m not synced with the server. Expected: 22, Header: 23 Delta deviation: 1


In [15]:
response

[[1726171320000,
  '58346.03000000',
  '58346.04000000',
  '58331.10000000',
  '58333.97000000',
  '5.20567000',
  1726171379999,
  '303681.22852830',
  536,
  '0.47340000',
  '27615.29355400',
  '0'],
 [1726171380000,
  '58333.98000000',
  '58357.71000000',
  '58326.00000000',
  '58326.00000000',
  '6.95488000',
  1726171439999,
  '405750.80820580',
  1063,
  '3.02271000',
  '176345.38245480',
  '0'],
 [1726171440000,
  '58326.01000000',
  '58345.28000000',
  '58326.00000000',
  '58342.37000000',
  '4.89518000',
  1726171499999,
  '285571.49403620',
  1439,
  '2.64701000',
  '154412.06675230',
  '0'],
 [1726171500000,
  '58342.37000000',
  '58344.00000000',
  '58282.00000000',
  '58326.15000000',
  '17.38638000',
  1726171559999,
  '1013865.23484630',
  3925,
  '6.36123000',
  '370917.81827930',
  '0'],
 [1726171560000,
  '58326.15000000',
  '58364.00000000',
  '58324.00000000',
  '58363.99000000',
  '12.45992000',
  1726171619999,
  '726977.67528910',
  2479,
  '10.03393000',
  '5854

In [16]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '48009',
 'Connection': 'keep-alive',
 'Date': 'Fri, 13 Sep 2024 12:41:24 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': '892ea37d-985a-4242-b7c6-5244be2bfcd8',
 'x-mbx-used-weight': '23',
 'x-mbx-used-weight-1m': '23',
 'content-encoding': 'gzip',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 b5e354980da78aef02917b4456c540c0.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'MAD56-P2',
 'X-Amz-Cf-Id': '8e3RvGzGt67KzVsJArA7

## Test order

In [17]:
url = 'https://api.binance.com/api/v3/order/test'
weight = 1

# timestamp is automatically added when signed call
params = {'symbol': "BTCUSDT",
          'side': "SELL",
          'type': "LIMIT",
          'timeInForce': 'GTC',
          'quantity': 0.001,
          'price': 80000,
          'recvWindow': 10000}

In [18]:
if l.can_make_request(weight=weight, is_order=True):
    response, headers = post(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(headers)

In [19]:
response

{}

In [20]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '2',
 'Connection': 'keep-alive',
 'Date': 'Fri, 13 Sep 2024 12:41:25 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': '7ae8abca-c744-4db0-b8be-d0efe297307e',
 'x-mbx-used-weight': '24',
 'x-mbx-used-weight-1m': '24',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 53ff0e7b87e36077aadcd25e028903a2.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'MAD56-P2',
 'X-Amz-Cf-Id': 'Iq4s90vvn3xa9VvYeie6O1EDHVCSLuhe0xFP9YDj2u5ojwn5mHXt2

## Trades

In [21]:
url = 'https://api.binance.com/api/v3/myTrades'
weight = 20
params = {
    'symbol': 'BTCUSDT',                   # The trading pair
    'limit': 3,                            # Optional: Limit the number of trades to retrieve (default 500)
    'recvWindow': 5000                     # Optional: Time window for the request (default 5000 ms)
}

In [22]:
if l.can_make_request(weight=weight, is_order=False):
    response, headers = get(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(headers)

In [23]:
response

[{'symbol': 'BTCUSDT',
  'id': 3315169490,
  'orderId': 23619145064,
  'orderListId': -1,
  'price': '41000.00000000',
  'qty': '0.00175000',
  'quoteQty': '71.75000000',
  'commission': '0.00023627',
  'commissionAsset': 'BNB',
  'time': 1702260789687,
  'isBuyer': True,
  'isMaker': True,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 3316660873,
  'orderId': 23687231478,
  'orderListId': -1,
  'price': '40986.19000000',
  'qty': '0.00128000',
  'quoteQty': '52.46232320',
  'commission': '0.00016332',
  'commissionAsset': 'BNB',
  'time': 1702320683140,
  'isBuyer': True,
  'isMaker': True,
  'isBestMatch': True},
 {'symbol': 'BTCUSDT',
  'id': 3327971249,
  'orderId': 23818929439,
  'orderListId': -1,
  'price': '43012.60000000',
  'qty': '0.00303000',
  'quoteQty': '130.32817800',
  'commission': '0.00039912',
  'commissionAsset': 'BNB',
  'time': 1702981648395,
  'isBuyer': False,
  'isMaker': False,
  'isBestMatch': True}]

In [24]:
headers

{'Content-Type': 'application/json;charset=UTF-8',
 'Content-Length': '311',
 'Connection': 'keep-alive',
 'Date': 'Fri, 13 Sep 2024 12:41:25 GMT',
 'Server': 'nginx',
 'x-mbx-uuid': '6b88702d-b2c3-4a18-87f0-fdc316f85d80',
 'x-mbx-used-weight': '44',
 'x-mbx-used-weight-1m': '44',
 'content-encoding': 'gzip',
 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Xss-Protection': '1; mode=block',
 'X-Content-Type-Options': 'nosniff',
 'Content-Security-Policy': "default-src 'self'",
 'X-Content-Security-Policy': "default-src 'self'",
 'X-WebKit-CSP': "default-src 'self'",
 'Cache-Control': 'no-cache, no-store, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'Access-Control-Allow-Origin': '*',
 'Access-Control-Allow-Methods': 'GET, HEAD, OPTIONS',
 'X-Cache': 'Miss from cloudfront',
 'Via': '1.1 306d2768a18200ca7fa9189a6c4470a6.cloudfront.net (CloudFront)',
 'X-Amz-Cf-Pop': 'MAD56-P2',
 'X-Amz-Cf-Id': '7JNPmP5MiYavu3fmJ8iU4j

In [25]:
l.get()

{'orders_limit_per_ten_seconds': 100,
 'orders_limit_per_day': 200000,
 'weight_limit_per_minute': 6000,
 'raw_limit_per_5_minutes': 61000,
 'server_time_offset': 928,
 'five_minutes_counts': {5754104: 5},
 'minutes_weights': {28770521: 44},
 'ten_seconds_orders_counts': {172623128: 1},
 'daily_orders_count': {19979: 1}}